In [16]:
from nustar_pysolar import planning, io
import astropy.units as u
from astropy.coordinates import SkyCoord
import warnings
warnings.filterwarnings('ignore')

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [10]:
#fname = io.download_occultation_times(outdir='../data/')
fname = '../data/NUSTAR.2020_253.SHADOW_ANALYSIS.txt'
print(fname)

../data/NUSTAR.2020_253.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [11]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [12]:
tstart = '2020-09-12T08:30:00'
tend = '2020-09-13T01:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

In [13]:
orbits

[[<Time object: scale='utc' format='isot' value=2020-09-12T09:04:40.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T10:06:00.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T10:41:20.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T11:42:40.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T12:18:00.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T13:19:20.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T13:54:40.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T14:55:50.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T15:31:20.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T16:32:30.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T17:08:00.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T18:09:10.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T18:44:40.000>,
  <Time object: scale='utc' form

In [14]:
# Get the solar parameter
from sunpy.coordinates import sun

angular_size = sun.angular_radius(t='now')
dx = angular_size.arcsec
print(dx)

966.9561774762417


In [20]:
offset = [-dx, 0]*u.arcsec
for ind, orbit in enumerate(orbits):
#    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
#    sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)

    
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos_beginning = planning.get_skyfield_position(orbit[0], offset, load_path='./data', parallax_correction=True)
    sky_pos_end = planning.get_skyfield_position(orbit[1], offset, load_path='./data', parallax_correction=True)

    pos1 = SkyCoord(sky_pos_beginning[0], sky_pos_beginning[1], unit='deg')
    pos2 = SkyCoord(sky_pos_end[0], sky_pos_end[1], unit='deg')
    
    print("Orbit: {}".format(ind))
#     print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
#     print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.3f} Dec (deg): {sky_pos[1]:8.3f}')
    

    print(pos1.separation(pos2).arcmin)
    
    
    print("")
    
    

Orbit: 0
2.2111582806337773

Orbit: 1
2.2112607739273686

Orbit: 2
2.2113651790011533

Orbit: 3
2.2040182473660463

Orbit: 4
2.204123214049978

Orbit: 5
2.2042300358707654

Orbit: 6
2.2043387164349113

Orbit: 7
2.21183089850589

Orbit: 8
2.2119406948504854

Orbit: 9
2.204597793498247



# This is where you actually make the Mosaic for Orbit 0

In [12]:
from importlib import reload
reload(planning)

<module 'nustar_pysolar.planning' from '/Users/bwgref/science/local/git/nustar_pysolar/build/lib.macosx-10.9-x86_64-3.7/nustar_pysolar/planning.py'>

In [14]:
pa = planning.get_nustar_roll(tstart, 0)
print(tstart)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

# We're actually using a SKY PA of 340. So...we'll need to rotate 
target_pa = 150
extra_roll = ( 150 - pa.value ) * u.deg
print(f'Extra roll used: {extra_roll}')


# Just use the first orbit...or choose one. This may download a ton of deltat.preds, which is a known 
# bug to be fixed.
orbit = orbits[0].copy()
print(orbit)
#...adjust the index above to get the correct orbit. Then uncomment below.

planning.make_mosaic(orbit, make_regions=True, extra_roll = extra_roll, outfile='orbit0_mosaic.txt', write_output=True)

2020-09-12T08:30:00
NuSTAR Roll angle for Det0 in NE quadrant: 23.630807678782805 deg
Extra roll used: 126.3691923212172 deg
[<Time object: scale='utc' format='isot' value=2020-09-12T09:04:40.000>, <Time object: scale='utc' format='isot' value=2020-09-12T10:06:00.000>]
Orbit start: 2020-09-12 09:04:40.000 Orbit end: 2020-09-12 10:06:00.000
('Dwell per position:', 147.19999999999973)

NuSTAR Roll Angle to get roll relative to Sun of 126.37 is 150.00 deg
Step of FOV PA direction is 240.00 deg

